# Google reviews scraping
## From a list of restaurants, scrap all the reviews and retrieve all users

Do reverse engineering. Start from the end. 

## Step 1: Get list of restaurant

In [282]:
import pandas as pd

df=pd.read_csv('data/restaurantsLausanne.csv')
lausanne_restaurants = df[["Company Name","City"]]
lausanne_restaurants[:5]


,Company Name,City
0,ARTISHOW Sàrl,Lausanne
1,Au 20 Sàrl,Lausanne
2,Holy Cow ! Gourmet Burger Company SA,Lausanne
3,Restaurant Osteria Balsamico Sàrl,Lausanne
4,Aperitivo Bar Restaurant Fuschetto & Vitale,Lausanne


In [283]:
df[:3]

,Zefix UID,Company Name,Legal Form,Capital Shares,Date of Registration,Address Line 1,Address Line 2,Street,House number,Zip,City,Canton,Website,Phone numbers,Email addresses,Physical Persons,NOGA Code,NOGA Description
0,276324973,ARTISHOW Sàrl,Société à responsabilité limitée,"20.000,00 CHF",07.04.17,NaN,c/o Andrea Probst,Avenue Gabriel-de-Rumine,40,1005,Lausanne,Vaud,NaN,NaN,NaN,"Loo van der,Associé et Gérant;Marc Lebel,Assoc...",561001.0,"Restaurants, cafés, snack-bar, tea-rooms et sa..."
1,113650633,Au 20 Sàrl,Société à responsabilité limitée,"50.000,00 CHF",01.06.07,NaN,NaN,Rue du Midi,20,1003,Lausanne,Vaud,www.midi20.ch,#ERROR!,info@midi20.ch,"Caroline Montandon-Varoda,Associée;Boris Karam...",NaN,NaN
2,114699895,Holy Cow ! Gourmet Burger Company SA,Société anonyme,"140.800,00 CHF",22.11.10,NaN,NaN,Rue des Terreaux,10,1003,Lausanne,Vaud,holycow.ch,NaN,events@holycow.ch,"Ryan Trask,Fondé de pouvoir;Adrian Stadelmann,...",561001.0,"Restaurants, cafés, snack-bar, tea-rooms et sa..."


## Step 2: Transform XHR file to json data

link example: https://www.google.com/async/reviewSort?vet=12ahUKEwign-avgeTeAhVCsaQKHcxNDkwQxyx6BAgBEAk..i&ved=2ahUKEwign-avgeTeAhVCsaQKHcxNDkwQjit6BAgBEFI&yv=3&async=feature_id:0x478c2fd32162af4b%3A0x292e2cc3a5d2256c,review_source:All%20reviews,sort_by:qualityScore,start_index:10,is_owner:false,filter_text:,next_page_token:,more:true,_pms:s,_fmt:pc

This file is messy. It needs to be parsed and transformed into a readable JSON file. 



In [79]:
# -*- coding: utf-8 -*-

from bs4 import BeautifulSoup
import sys 
import json


reviews = []
soup = BeautifulSoup(open("data/f.txt", 'r'), 'html.parser')
#print(soup)
for review in soup.find_all('div', attrs={'class': 'WMbnJf'}):
    r = {}
    wrapper = review.find('div', attrs={'class':'TSUbDb'})
    author = wrapper.find('a', attrs={'class': 'Y0uHMb'})
    author_link = author['href']
    author_name = author.text
    
    grade = int(review.find('span', attrs={'class':'Fam1ne'})['aria-label'][7])
    review_text = review.find('div', attrs={'class':'Jtu6Td'}).findChildren('span')[0].text
    
    r['author_name'] = author_name
    r['author_link'] = author_link
    r['grade'] = grade
    r['review'] = review_text
    reviews.append(r)

print(json.dumps(reviews[:4], indent=2,ensure_ascii=False))
#reviews_json = json.dumps(reviews,ensure_ascii=False)
#with open('reviews1.json', 'w') as outfile:
#    json.dump(reviews, outfile,ensure_ascii=False)

[
  {
    "author_name": "Julien Schnetzer",
    "grade": 5,
    "author_link": "https://www.google.com/maps/contrib/109651076030094729306/reviews?hl=fr-CH&sa=X&ved=2ahUKEwivvuzbguTeAhWQGuwKHRNABF8QvvQBegQIARAD",
    "review": "Rien a redire super experience et service de qualité"
  },
  {
    "author_name": "Olivier Vermeulen",
    "grade": 4,
    "author_link": "https://www.google.com/maps/contrib/101544869006984350636/reviews?hl=fr-CH&sa=X&ved=2ahUKEwivvuzbguTeAhWQGuwKHRNABF8QvvQBegQIARAK",
    "review": "Accueil sympathique idéal pour prendre un café et pour lire son quotidien(imprimé un luxe qui se perds)"
  },
  {
    "author_name": "François Collet",
    "grade": 5,
    "author_link": "https://www.google.com/maps/contrib/105006659743746115145/reviews?hl=fr-CH&sa=X&ved=2ahUKEwivvuzbguTeAhWQGuwKHRNABF8QvvQBegQIARAR",
    "review": "Restaurant exceptionnel!Cuisine irréprochable à l'Italienne!Patron jovial et personnel adorable!"
  },
  {
    "author_name": "Francisco Kolb",
    "gr

## Step 3: Same as step 2, but get data from link

The problem is that when you click on the link:  https://www.google.com/async/reviewSort?vet=12ahUKEwign-avgeTeAhVCsaQKHcxNDkwQxyx6BAgBEAk..i&ved=2ahUKEwign-avgeTeAhVCsaQKHcxNDkwQjit6BAgBEFI&yv=3&async=feature_id:0x478c2fd32162af4b%3A0x292e2cc3a5d2256c,review_source:All%20reviews,sort_by:qualityScore,start_index:10,is_owner:false,filter_text:,next_page_token:,more:true,_pms:s,_fmt:pc -> it makes you download the file. Only after having downloaded the file, you can parse it.

The goal here is to get the data from the link directly. 

In [136]:
# -*- coding: utf-8 -*-

from bs4 import BeautifulSoup
import sys 
import json
import requests

link = "https://www.google.com/async/reviewSort?vet=12ahUKEwign-avgeTeAhVCsaQKHcxNDkwQxyx6BAgBEAk..i&ved=2ahUKEwign-avgeTeAhVCsaQKHcxNDkwQjit6BAgBEFI&yv=3&async=feature_id:0x478c2fd32162af4b%3A0x292e2cc3a5d2256c,review_source:All%20reviews,sort_by:qualityScore,start_index:10,is_owner:false,filter_text:,next_page_token:,more:true,_pms:s,_fmt:pc"
link2 = "https://www.google.com/async/reviewSort?vet=12ahUKEwiigKqTleTeAhXDDOwKHcYoBHoQxyx6BAgBEAk..i&ved=2ahUKEwiigKqTleTeAhXDDOwKHcYoBHoQjit6BAgBEFI&yv=3&async=feature_id:0x478c2e34537fa595%3A0xb4c99c55735dab0c,review_source:All%20reviews,sort_by:qualityScore,start_index:10,is_owner:false,filter_text:,next_page_token:,more:true,_pms:s,_fmt:pc"

headers = {
    'accept-language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7',
    'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2272.101 Safari/537.36'

}
    
r = requests.get(link, headers=headers)

soup = BeautifulSoup(r.text, 'html.parser')
reviews = []
for review in soup.find_all('div', attrs={'class': 'WMbnJf'}):
    r = {}
    wrapper = review.find('div', attrs={'class':'TSUbDb'})
    author = wrapper.find('a', attrs={'class': 'Y0uHMb'})
    author_link = author['href']
    author_name = author.text
    
    grade = int(review.find('span', attrs={'class':'Fam1ne'})['aria-label'][7])
    review_text = review.find('div', attrs={'class':'Jtu6Td'}).findChildren('span')[0].text
    
    r['author_name'] = author_name
    r['author_link'] = author_link
    r['grade'] = grade
    r['review'] = review_text
    reviews.append(r)

print(json.dumps(reviews[:4], indent=2, ensure_ascii=False)) 




[
  {
    "author_name": "Julien Schnetzer",
    "grade": 5,
    "author_link": "https://www.google.com/maps/contrib/109651076030094729306/reviews?hl=fr-CH&sa=X&ved=2ahUKEwjQhYHXx-beAhUD3qQKHc69AmUQvvQBegQIARAD",
    "review": "Rien a redire super experience et service de qualité"
  },
  {
    "author_name": "Olivier Vermeulen",
    "grade": 4,
    "author_link": "https://www.google.com/maps/contrib/101544869006984350636/reviews?hl=fr-CH&sa=X&ved=2ahUKEwjQhYHXx-beAhUD3qQKHc69AmUQvvQBegQIARAK",
    "review": "Accueil sympathique idéal pour prendre un café et pour lire son quotidien(imprimé un luxe qui se perds)"
  },
  {
    "author_name": "François Collet",
    "grade": 5,
    "author_link": "https://www.google.com/maps/contrib/105006659743746115145/reviews?hl=fr-CH&sa=X&ved=2ahUKEwjQhYHXx-beAhUD3qQKHc69AmUQvvQBegQIARAR",
    "review": "Restaurant exceptionnel!Cuisine irréprochable à l'Italienne!Patron jovial et personnel adorable!"
  },
  {
    "author_name": "Francisco Kolb",
    "gr

## Step 4: Make a function out of the parser, with the link as input

In [175]:
# -*- coding: utf-8 -*-

from bs4 import BeautifulSoup
import sys 
import requests
import json


def ftxt_clean(link):
    headers = {
        'accept-language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7',
        'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2272.101 Safari/537.36'

    }
    r = requests.get(link, headers=headers)
    soup = BeautifulSoup(r.text,'html.parser')
    reviews = []
    for review in soup.find_all('div', attrs={'class': 'WMbnJf'}):
        r = {}
        wrapper = review.find('div', attrs={'class':'TSUbDb'})
        author = wrapper.find('a', attrs={'class': 'Y0uHMb'})
        author_link = author['href']
        author_name = author.text

        grade = int(review.find('span', attrs={'class':'Fam1ne'})['aria-label'][7])
        review_text = review.find('div', attrs={'class':'Jtu6Td'}).findChildren('span')[0].text

        r['author_name'] = author_name
        r['author_link'] = author_link
        r['grade'] = grade
        r['review'] = review_text
        reviews.append(r)
    return reviews

def clean_xhr_from_soup(soup):
    reviews = []
    for review in soup.find_all('div', attrs={'class': 'WMbnJf'}):
        r = {}
        wrapper = review.find('div', attrs={'class':'TSUbDb'})
        author = wrapper.find('a', attrs={'class': 'Y0uHMb'})
        author_link = author['href']
        author_name = author.text

        grade = int(review.find('span', attrs={'class':'Fam1ne'})['aria-label'][7])
        review_text = review.find('div', attrs={'class':'Jtu6Td'}).findChildren('span')[0].text

        r['author_name'] = author_name
        r['author_link'] = author_link
        r['grade'] = grade
        r['review'] = review_text
        reviews.append(r)
    return reviews
    
    
link = "https://www.google.com/async/reviewSort?vet=12ahUKEwign-avgeTeAhVCsaQKHcxNDkwQxyx6BAgBEAk..i&ved=2ahUKEwign-avgeTeAhVCsaQKHcxNDkwQjit6BAgBEFI&yv=3&async=feature_id:0x478c2fd32162af4b%3A0x292e2cc3a5d2256c,review_source:All%20reviews,sort_by:qualityScore,start_index:10,is_owner:false,filter_text:,next_page_token:,more:true,_pms:s,_fmt:pc"
reviews_json = ftxt_clean(link)
print(json.dumps(reviews_json[:2],indent=2,ensure_ascii=False))


[
  {
    "author_name": "Julien Schnetzer",
    "grade": 5,
    "author_link": "https://www.google.com/maps/contrib/109651076030094729306/reviews?hl=fr-CH&sa=X&ved=2ahUKEwi2vsiy1ebeAhUN-aQKHXfXBksQvvQBegQIARAD",
    "review": "Rien a redire super experience et service de qualité"
  },
  {
    "author_name": "Olivier Vermeulen",
    "grade": 4,
    "author_link": "https://www.google.com/maps/contrib/101544869006984350636/reviews?hl=fr-CH&sa=X&ved=2ahUKEwi2vsiy1ebeAhUN-aQKHXfXBksQvvQBegQIARAK",
    "review": "Accueil sympathique idéal pour prendre un café et pour lire son quotidien(imprimé un luxe qui se perds)"
  }
]


## Create function for request and get parsed data.


In [176]:
def get_soup_from_link(link):
    headers = {
        'accept-language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7',
        'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2272.101 Safari/537.36'

    }
    r = requests.get(link, headers=headers)
    soup = BeautifulSoup(r.text,'html.parser')
    return soup

def decompose_link(link):
    splitted_link = link.split('start_index:')
    return [splitted_link[0]+'start_index:', splitted_link[1][1:]]

# Step 5: Get all reviews for one restaurant
## 5.1 : Get the number of reviews

The best way to get the number of reviews is to get the link of the file at index=0. In this file, it is easy to retrieve the number of reviews. 

First, we'll just get the information of the number of reviews, then focus on automatically get the link of this file. 

In [153]:
link_boucanier0 = 'https://www.google.com/async/reviewDialog?ei=SeL1W5CeNu3srgSoi4po&yv=3&async=feature_id:0x478c2e34537fa595%3A0xb4c99c55735dab0c,review_source:All%20reviews,sort_by:qualityScore,start_index:0,is_owner:false,filter_text:,next_page_token:,async_id_prefix:,_pms:s,_fmt:pc'
link_milan0 = 'https://www.google.com/async/reviewDialog?ei=_-T1W8v-AoGrsAH9sojwDw&yv=3&async=feature_id:0x478c2fd32162af4b%3A0x292e2cc3a5d2256c,review_source:All%20reviews,sort_by:qualityScore,start_index:0,is_owner:false,filter_text:,next_page_token:,async_id_prefix:,_pms:s,_fmt:pc'
def get_number_of_reviews(link):
    soup = get_parsed_request(link)
    number_of_reviews = int(soup.find('span', attrs={'class':'Vfp4xe'}).text.split()[0])
    return number_of_reviews

def get_number_of_reviews_from_soup(soup):
    return int(soup.find('span', attrs={'class':'Vfp4xe'}).text.split()[0])
                                  
number_of_rev = get_number_of_reviews(link_milan0)
number_of_rev

476

## 5.2: Retrieve all the reviews

Write a function that loops into all the files and retrieve all the reviews.. As input data, the first link of the XHR file of the restaurant.

In [264]:
link_milan0 = 'https://www.google.com/async/reviewDialog?ei=_-T1W8v-AoGrsAH9sojwDw&yv=3&async=feature_id:0x478c2fd32162af4b%3A0x292e2cc3a5d2256c,review_source:All%20reviews,sort_by:qualityScore,start_index:0,is_owner:false,filter_text:,next_page_token:,async_id_prefix:,_pms:s,_fmt:pc'
link_lipp0 = 'https://www.google.com/async/reviewDialog?ei=m_X1W-jiHIzQrgSI1KqoCg&yv=3&async=feature_id:0x478c65296a2d20a5%3A0xfea688b790ace557,review_source:All%20reviews,sort_by:qualityScore,start_index:0,is_owner:false,filter_text:,next_page_token:,async_id_prefix:,_pms:s,_fmt:pc'

def get_all_the_reviews_of_restaurant_from_xhr_link(link):
    soup = get_parsed_request(link)
    number_of_reviews = get_number_of_reviews_from_soup(soup)
    last_index = number_of_reviews - (number_of_reviews % 10)
    index_zero = 0
    splitted_link = decompose_link(link)
    all_reviews = []
    for index in range(index_zero,last_index+1,10):
        new_link = splitted_link[0]+str(index)+splitted_link[1]
        soup = get_soup_from_link(new_link)
        json = clean_xhr_from_soup(soup)
        all_reviews = all_reviews + json
    return all_reviews
        
        

#all_reviews = get_all_the_reviews_of_restaurant(link_lipp0)


In [195]:
with open('reviews.json', 'w') as outfile:
    json.dump(all_reviews, outfile,ensure_ascii=False)

# Step 6: Get the link to index 0 automatically from dialog link

Done by decomposing the link and search these informations in the html response.

In [258]:
link_to_dialog = "https://www.google.com/search?q=pocco+loco+lausanne&oq=pocco+loco+lausanne&aqs=chrome..69i57.3060j0j9&sourceid=chrome&ie=UTF-8#lrd=0x478c2e2ebc73431b:0x49212fbfc2d564fb,1,,,"
headers = {
    'accept-language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7',
    'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2272.101 Safari/537.36'

}
r = requests.get(link_to_dialog, headers=headers)
soup = BeautifulSoup(r.text,'html.parser')
ei = soup.find('div',attrs={'id':'tophf'}).findChildren('input',attrs={'name':'ei'})[0]['value']
feature_id_all = soup.find('a', attrs={'class': 'YIuV9'})['data-async-context']
feature_id = feature_id_all.split('feature_id:')[1][:39]


original_link = "https://www.google.com/async/reviewDialog?ei=k4n4W7bfC4fJsQHOmpbgCQ&yv=3&async=feature_id:0x478c2fd32162af4b%3A0x292e2cc3a5d2256c,review_source:All%20reviews,sort_by:qualityScore,start_index:0,is_owner:false,filter_text:,next_page_token:,async_id_prefix:,_pms:s,_fmt:pc"
link_auto = "https://www.google.com/async/reviewDialog?ei="+ei+"&yv=3&async=feature_id:"+feature_id+",review_source:All%20reviews,sort_by:qualityScore,start_index:0,is_owner:false,filter_text:,next_page_token:,async_id_prefix:,_pms:s,_fmt:pc"


## Create function out of step 6

In [259]:
def retrieve_xhr_link_from_dialog_link(link_to_dialog):
    headers = {
        'accept-language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7',
        'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2272.101 Safari/537.36'
    }
    r = requests.get(link_to_dialog, headers=headers)
    soup = BeautifulSoup(r.text,'html.parser')
    ei = soup.find('div',attrs={'id':'tophf'}).findChildren('input',attrs={'name':'ei'})[0]['value']
    feature_id_all = soup.find('a', attrs={'class': 'YIuV9'})['data-async-context']
    feature_id = feature_id_all.split('feature_id:')[1][:39]
    return "https://www.google.com/async/reviewDialog?ei="+ei+"&yv=3&async=feature_id:"+feature_id+",review_source:All%20reviews,sort_by:qualityScore,start_index:0,is_owner:false,filter_text:,next_page_token:,async_id_prefix:,_pms:s,_fmt:pc"



# Step 7: Get file link from restaurant + town google query

In [263]:
restaurant = "milan"
town = "lausanne"

google_query="https://www.google.com/search?q=pocco+loco+lausanne"
headers = {
    'accept-language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7',
    'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2272.101 Safari/537.36'
}
r = requests.get(google_query, headers=headers)
soup = BeautifulSoup(r.text,'html.parser')
ei = soup.find('div',attrs={'id':'tophf'}).findChildren('input',attrs={'name':'ei'})[0]['value']
feature_id = soup.find('a', attrs={'class': 'YIuV9'})['data-async-context'].split('feature_id:')[1][:39]


In [291]:
def retrieve_xhr_link_from_restaurant_town_query(restaurant,town):
    google_query = "https://www.google.com/search?q=restaurant+"+restaurant+"+"+town
    print(google_query)
    headers = {
        'accept-language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7',
        'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2272.101 Safari/537.36'
    }
    r = requests.get(google_query, headers=headers)
    soup = BeautifulSoup(r.text,'html.parser')
    ei = soup.find('div',attrs={'id':'tophf'}).findChildren('input',attrs={'name':'ei'})[0]['value']
    feature_id = soup.find('a', attrs={'class': 'YIuV9'})['data-async-context'].split('feature_id:')[1][:39]
    return "https://www.google.com/async/reviewDialog?ei="+ei+"&yv=3&async=feature_id:"+feature_id+",review_source:All%20reviews,sort_by:qualityScore,start_index:0,is_owner:false,filter_text:,next_page_token:,async_id_prefix:,_pms:s,_fmt:pc"


## Function: Get all reviews for a restaurant from restaurant + town query

In [272]:
def get_all_reviews_from_restaurant_town(restaurant, town):
    xhr_link = retrieve_xhr_link_from_restaurant_town_query(restaurant,town)
    reviews = get_all_the_reviews_of_restaurant_from_xhr_link(xhr_link)
    return reviews

reviews = get_all_reviews_from_restaurant_town(lausanne_restaurant, "lausanne")

NameError: name 'lausanne_restaurant' is not defined

In [298]:
restaurant = lausanne_restaurants['Company Name'][0]
city = lausanne_restaurants['City'][0]

reviews = get_all_reviews_from_restaurant_town("molino","vevey")
reviews[:5]

https://www.google.com/search?q=restaurant+molino+vevey


[{'author_link': 'https://www.google.com/maps/contrib/114257953594334509855/reviews?hl=fr-CH&sa=X&ved=2ahUKEwifpvzTx_DeAhWEjqQKHS5KDFoQvvQBegQIARAN',
  'author_name': 'Jacques Blote',
  'grade': 4,
  'review': "Excellentes pizzas déclinées sous 2 manières. Bon service. En général il est bon de réserver car à 12h30, l'établissement est plein!! Bon choix de vins."},
 {'author_link': 'https://www.google.com/maps/contrib/105084559907226706902/reviews?hl=fr-CH&sa=X&ved=2ahUKEwifpvzTx_DeAhWEjqQKHS5KDFoQvvQBegQIARAU',
  'author_name': 'Anna Ferreira',
  'grade': 5,
  'review': 'Ils font de très bonnes pizzas, même nos amis italiens ont apprécié. Le week-end il faut réserver pour le soir, car le restaurant se remplit vite. Les pâtes sont aussi un régal !!!'},
 {'author_link': 'https://www.google.com/maps/contrib/103593497845798328644/reviews?hl=fr-CH&sa=X&ved=2ahUKEwifpvzTx_DeAhWEjqQKHS5KDFoQvvQBegQIARAb',
  'author_name': 'Gisèle Umuhoza',
  'grade': 5,
  'review': "BEST PIZZA PLACE IN TOWN! 

In [299]:
reviews

[{'author_link': 'https://www.google.com/maps/contrib/114257953594334509855/reviews?hl=fr-CH&sa=X&ved=2ahUKEwifpvzTx_DeAhWEjqQKHS5KDFoQvvQBegQIARAN',
  'author_name': 'Jacques Blote',
  'grade': 4,
  'review': "Excellentes pizzas déclinées sous 2 manières. Bon service. En général il est bon de réserver car à 12h30, l'établissement est plein!! Bon choix de vins."},
 {'author_link': 'https://www.google.com/maps/contrib/105084559907226706902/reviews?hl=fr-CH&sa=X&ved=2ahUKEwifpvzTx_DeAhWEjqQKHS5KDFoQvvQBegQIARAU',
  'author_name': 'Anna Ferreira',
  'grade': 5,
  'review': 'Ils font de très bonnes pizzas, même nos amis italiens ont apprécié. Le week-end il faut réserver pour le soir, car le restaurant se remplit vite. Les pâtes sont aussi un régal !!!'},
 {'author_link': 'https://www.google.com/maps/contrib/103593497845798328644/reviews?hl=fr-CH&sa=X&ved=2ahUKEwifpvzTx_DeAhWEjqQKHS5KDFoQvvQBegQIARAb',
  'author_name': 'Gisèle Umuhoza',
  'grade': 5,
  'review': "BEST PIZZA PLACE IN TOWN! 